In [1]:
import os


# os.environ["CUDA_VISIBLE_DEVICES"] = ""
device = "cuda"


In [2]:
# 80
import pandas as pd


train = pd.read_table("train.tsv")
valid = pd.read_table("valid.tsv")
test = pd.read_table("test.tsv")

In [3]:
from itertools import chain

vocab = chain.from_iterable(train["title"].str.split())
vocab = pd.Series(vocab, name="word")
vocab = vocab.value_counts().reset_index(name="count")
vocab = vocab.loc[vocab["count"] >= 2, "word"]
vocab = {w: i for i, w in enumerate(vocab.tolist(), start=1)}
assert "<unk>" not in vocab
vocab["<unk>"] = 0
len(vocab)

10347

In [4]:
from dataset import Tokenier

tokenizer = Tokenier(vocab)
tokenizer("hello world This is a pen you are kind")


[0, 1367, 104, 32, 18, 0, 498, 78, 0]

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
len(vocab)

10347

In [7]:
max(vocab.values())

10346

In [8]:
# 81
import torch
from models import RNN

model = RNN(vocab_size=len(vocab), hidden_size=300, output_size=5)
model(torch.tensor([[1, 2, 3]]),labels=torch.tensor([3]))


ModelOutput([('loss', tensor(1.5940, grad_fn=<NllLossBackward0>)),
             ('logits',
              tensor([[0.1585, 0.2596, 0.1782, 0.2160, 0.1877]], grad_fn=<SoftmaxBackward0>))])

In [23]:
# 82
# TODO: trainer に移動
from sklearn.metrics import accuracy_score, f1_score
from transformers import EvalPrediction


def custom_compute_metrics(res: EvalPrediction) -> dict:
    pred = res.predictions.argmax(axis=1)
    acc = accuracy_score(res.label_ids, pred)
    f1 = f1_score(res.label_ids, pred, average="macro")
    return {"accuracy": acc, "f1": f1}


In [16]:
from transformers import TrainingArguments
from dataset import NewsAggCollator, NewsAggDataset
from transformers.trainer import Trainer


training_args = TrainingArguments(
    output_dir="/data/yuji96/nlp100/82_rnn_sgd",
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="f1",
    load_best_model_at_end=True,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=15,
    remove_unused_columns=False,
    report_to="wandb",
)


PyTorch: setting up devices


In [24]:
from transformers import EarlyStoppingCallback

model = model.to(device)
collator = NewsAggCollator(tokenizer, device=device)

sgd = torch.optim.SGD(model.parameters(), lr=0.001)
# scheduler = torch.optim.lr_scheduler.ConstantLR(sgd, factor=1, total_iters=1)
scheduler = None
trainer = Trainer(
    model,
    training_args,
    train_dataset=NewsAggDataset(train),
    eval_dataset=NewsAggDataset(valid),
    data_collator=collator,
    compute_metrics=custom_compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
    optimizers=(sgd, scheduler),
)
trainer.train()


***** Running training *****
  Num examples = 10672
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 96
  Gradient Accumulation steps = 1
  Total optimization steps = 1120
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
/home/yuji96/.conda/envs/yuji/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss


***** Running Evaluation *****
  Num examples = 1334
  Num examples = 1334
Saving model checkpoint to /data/yuji96/nlp100/82_rnn_sgd/checkpoint-112
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num examples = 1334
  Batch size = 32
Saving model checkpoint to /data/yuji96/nlp100/82_rnn_sgd/checkpoint-224
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num examples = 1334
  Batch size = 32
Saving model checkpoint to /data/yuji96/nlp100/82_rnn_sgd/checkpoint-336
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num examples = 1334
  Batch size = 32
Saving model checkpoint to /data/yuji96/nlp100/82_rnn_sgd/checkpoint-448
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num examples = 1334
  Batch size = 32
***** Running Evaluation *****
  Num examples = 1334
  Batch size = 3

TrainOutput(global_step=1120, training_loss=1.6465275355747768, metrics={'train_runtime': 149.7769, 'train_samples_per_second': 712.527, 'train_steps_per_second': 7.478, 'total_flos': 0.0, 'train_loss': 1.6465275355747768, 'epoch': 10.0})